# Neutronics simulation with CAD geometry

This example creates a CAD geometry and then carries out a neutronics simulation with a heating cell tally.

This section makes a few components and places them in a reactor object. These will form the CAD model with which we will perform a neutronics simulation.

In [ ]:
import paramak

pf_1 = paramak.PoloidalFieldCoil(
    height=50,
    width=50,
    center_point=(800,50),
    rotation_angle=360,  # note the components are all rotated 360 to make a full sector model
    material_tag='pf_coil_mat'
)

tf = paramak.ToroidalFieldCoilCoatHanger(
    horizontal_start_point=(200, 500),
    horizontal_length=400,
    vertical_mid_point=(700, 50),
    vertical_length=500,
    thickness=50,
    distance=50,
    stp_filename="toroidal_field_coil_coat_hanger.stp",
    number_of_coils=8,
    with_inner_leg=True,
    material_tag='tf_coil_mat'
)


plasma = paramak.Plasma(
    minor_radius=150.,
    major_radius=450.,
    triangularity=0.55,
    elongation=2.,
    rotation_angle=360
)

blanket = paramak.BlanketFP(
    plasma=plasma,
    thickness=20,
    stop_angle=90,
    start_angle=-90,
    offset_from_plasma=20,
    rotation_angle=360,
    material_tag='blanket_mat'
)

my_reactor = paramak.Reactor([pf_1, tf, blanket])  # note the plasma is not added
#  The plasma doesn't interact much with the neutrons (low density)
# but the curved shape does slow down the simulation due to the large number of facets need
# so it has been removed to speed up the example

my_reactor.rotation_angle=360  # this is a small "feature" which we will fix soon
my_reactor.show()

This section makes a plasma source which we will use as the source for the neutronics model.

In [ ]:
import openmc

# initialises a new source object
source = openmc.Source()
# the distribution of radius is just a single value
radius = openmc.stats.Discrete([300], [1])
# the distribution of source z values is just a single value
z_values = openmc.stats.Discrete([0], [1])
# the distribution of source azimuthal angles values is a uniform distribution between 0 and 2 Pi
angle = openmc.stats.Uniform(a=0., b=2* 3.14159265359)
# this makes the ring source using the three distributions and a radius
source.space = openmc.stats.CylindricalIndependent(r=radius, phi=angle, z=z_values, origin=(0.0, 0.0, 0.0))
# sets the direction to isotropic
source.angle = openmc.stats.Isotropic()
# sets the energy distribution to a Muir distribution neutrons
source.energy = openmc.stats.Muir(e0=14080000.0, m_rat=5.0, kt=20000.0)

This section forms the neutronics model by combining the 3D model, the plasma source and some assigned materials. Additionally, the tallies to record the heating are specified. There are a few different methods of converting a CAD model into a neutronics model. The 'pymoab' method is the only open-source automated method that can also convert splines.

In [ ]:
import paramak_neutronics

neutronics_model = paramak_neutronics.NeutronicsModel(
    geometry=my_reactor,
    cell_tallies=['heating'],
    source=source,
    simulation_batches=10,
    simulation_particles_per_batch=10,
    materials={
        'pf_coil_mat': 'copper',
        'tf_coil_mat': 'copper',
        'blanket_mat': 'Li4SiO4',
    }
)

neutronics_model.simulate()

The results of the simulation are printed with the following command. The tallied neutronics output was heating.

In [ ]:
neutronics_model.results

**Learning Outcomes from Task 11**

- Neutronics simulations can be performed by combining CAD models, neutron sources and material definitions.